***GENERATED CODE FOR delete PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('total_beds_7_day_sum')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run deleteHooks.ipynb
try:
	#sourcePreExecutionHook()

	tetctcu = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-6d3e44e9-d955-4613-bd15-4f5c6a348b5e-c000.csv', 'filename': '1718007076Inputfile.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/TimeSeries/CoronaTS/datafile/test.csv', 'viewFileName': 'test.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run deleteHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(tetctcu,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "collection_week", "transformation_label": "novalue"}], "feature": "collection_week", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "collection_week"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "novalue"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "37", "mean": "", "stddev": "", "min": "AR", "max": "TX", "missing": "0", "distinct": "22"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "novalue"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "37", "mean": "0.0", "stddev": "nan", "min": "0", "max": "WOODLAND HILLS", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "hospital_subtype", "transformation_label": "novalue"}], "feature": "hospital_subtype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "37", "mean": "", "stddev": "", "min": "Critical Access Hospitals", "max": "Short Term", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hospital_subtype"}, {"transformationsData": [{"feature_label": "fips_code", "threshold": 27791.404312668463, "transformation_label": "novalue"}], "feature": "fips_code", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "25624.59", "stddev": "15063.65", "min": "0.0", "max": "48423.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fips_code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_metro_micro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "0.84", "stddev": "0.37", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "is_metro_micro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "178.57", "stddev": "253.99", "min": "0.0", "max": "913.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_avg"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "15.84", "stddev": "25.78", "min": "0.0", "max": "97.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_sum", "type": "real", "selected": "False", "replaceby": "mean", "stats": {"count": "37", "mean": "961.68", "stddev": "1470.12", "min": "0.0", "max": "6391.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "274.97", "stddev": "796.12", "min": "0.0", "max": "4177.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_adult_hospital_beds_7..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_inpatient_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "205.54", "stddev": "661.9", "min": "0.0", "max": "3519.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "all_adult_hospital_inpati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "102.54", "stddev": "175.75", "min": "0.0", "max": "684.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "32.57", "stddev": "98.49", "min": "0.0", "max": "527.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "21.86", "stddev": "66.94", "min": "0.0", "max": "369.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "785.65", "stddev": "1114.31", "min": "0.0", "max": "5060.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_icu_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "112.89", "stddev": "220.39", "min": "0.0", "max": "1192.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_icu_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_staffed_adult_icu_beds_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "32.62", "stddev": "97.6", "min": "0.0", "max": "490.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_staffed_adult_icu_b..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "icu_beds_used_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "27.81", "stddev": "82.24", "min": "0.0", "max": "396.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "icu_beds_used_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "3.81", "stddev": "12.51", "min": "0.0", "max": "65.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_covid_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "3.19", "stddev": "10.54", "min": "0.0", "max": "54.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_covid_ED_visits_7_day_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "37", "mean": "28.7", "stddev": "88.48", "min": "0.0", "max": "454.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "previous_day_covid_ED_vis..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run deleteHooks.ipynb
try:
	#sinkPreExecutionHook()

	datafileoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/6666c10db8756b8967bd27dd/part-00000-263f7539-b08a-458d-b39c-3276e3374c4d-c000.csv', 'samplecount': 371, 'originalcount': 371, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/part-00000-6d3e44e9-d955-4613-bd15-4f5c6a348b5e-c000.csv', 'pathOnly': '/TimeSeries/CoronaTS/datafile', 'project_id': '6666a69bb8756b8967bd1570', 'parent_id': '6666a69bb8756b8967bd1570', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2934', 'field': 'state', 'alias': 'state', 'type': 'string', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2935', 'field': 'city', 'alias': 'city', 'type': 'string', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2936', 'field': 'hospital_subtype', 'alias': 'hospital_subtype', 'type': 'string', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2937', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2938', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2939', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293a', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293b', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293c', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293d', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293e', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd293f', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2940', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2941', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2942', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2943', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2944', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2945', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2946', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'real', 'position': '18'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd291f', 'field': 'collection_week', 'alias': 'collection_week', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2920', 'field': 'state', 'alias': 'state', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2921', 'field': 'city', 'alias': 'city', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2922', 'field': 'hospital_subtype', 'alias': 'hospital_subtype', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2923', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2924', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2925', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2926', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2927', 'field': 'total_beds_7_day_sum', 'alias': 'total_beds_7_day_sum', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2928', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2929', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292a', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292b', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292c', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292d', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292e', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd292f', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2930', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2931', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2932', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '6666c207b8756b8967bd2933', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'real', 'position': '20'}], 'server': 'https://dnm.bfirst.ai:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'test.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-01b3c6ca-471b-494d-be19-f594ec1c5deb-c000.csv', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
